In [3]:
"""
Created on Sun Jun 20 18:53:11 2021

@author: Johanna
"""

import pandas as pd
import numpy as np 
import glob

twitter_list = []

path = r'./Nodes' # use your path
all_files = glob.glob(path + "/*.parquet")

for filename in all_files:
    df = pd.read_parquet(filename, engine='pyarrow')
    twitter_list.append(df)
    
#twitter = pd.read_parquet('nodes_1.parquet', engine='pyarrow')
twitter = pd.concat(twitter_list)

#for col in twitter.columns:
#    print(col)

# absolute values: followers
# six honest signals based on chosen values: check if correct values were chosen from the data 
# central leadership: degree centrality, betweenness centrality
# rotation leadership: betweenness centrality oscillation
# balanced contribution: contribution index
# rapid response: ego art, ego nudges, alter art, alter nudges
# honest language: avg. sentiment, avg. emotionality
# shared context: avg. complexity 


twitter = twitter[['id', 'name', 'followers_count', 'degree', 'betweenness', 'contribution_index', 
                   'ego_art', 'ego_nudges', 'alter_art', 'alter_nudges', 'sentiment_avg', 'emotionality_avg',
                   'complexity_avg']]

twitter = twitter.fillna(0)

xls = pd.ExcelFile('Intelektuellen-Ranking.xlsx')

df = xls.parse(parse_cols=[0, 1, 2, 3], index_col=0, nrows=100, usecols = "A,B,C,D,E,F,G", na_values=[0])

#print(df["Twitter_id"].value_counts())

print(df)

# merging the datasets
twitter = twitter.rename(columns={'name' : 'Name'})
combined = pd.merge(twitter, df, left_on='id', right_on="Twitter_id")

combined = combined.drop(columns=['Bereich', 'Wikipedia', 'Google Search Results'])

conditions = [
    (combined['Twitter Verifiziert?'] == 'nicht verifiziert') | (combined['Twitter Verifiziert?'] == 0),
    (combined['Twitter Verifiziert?'] == 'verifiziert')]

values = [0, 1]

combined['verified'] = np.select(conditions, values)

# calculations for the values we need 

combined['central_leadership'] = combined['degree'] + combined['betweenness']
#combined['rotation_leadership'] = combined['betweenness_oscillation']
combined['balanced_contribution'] = combined['contribution_index']
combined['rapid_responses'] = (1/combined['ego_art']) + (1/combined['ego_nudges']) + (1/combined['alter_nudges']) + (1/combined['alter_art'])
combined['honest_language'] = combined['sentiment_avg'] + combined['emotionality_avg']
combined['shared_context'] = combined['complexity_avg']
#combined = combined.fillna(0)
combined['Twitter'] = combined['central_leadership'] + combined['balanced_contribution'] + (1/combined['rapid_responses']) + combined['honest_language'] + combined['shared_context']+(combined['followers_count']/100)

#drop unncessary columns 
combined = combined[['Name', 'Twitter']]

def get_twitter_factor(): 
    global twitter_factor; 
    twitter_factor = pd.DataFrame(combined)
    #twitter_factor.sort_values(by=["twitter_index"], inplace=True)
    
    
    twitter_factor['Twitter']=(twitter_factor['Twitter']-twitter_factor['Twitter'].min())/(twitter_factor['Twitter'].max()-twitter_factor['Twitter'].min())
    return twitter_factor;

get_twitter_factor()

                     m/w      Bereich             Wikipedia       Twitter_id  \
Name                                                                           
Adolf Muschg          m     Literatur          Adolf Muschg                -   
Aleksanra Bachzetsis   w        Kunst  Alexandra Bachzetsis                -   
Andreas Vollenweider   m        Musik  Andreas Vollenweider   a_vollenweider   
Anna Rosenwasser       w   Aktivismus                     -  AnnaRosenwasser   
Annemarie Piper        w  Philosophie      Annemarie Pieper                -   
...                   ..          ...                   ...              ...   
Tommy Vercetti         m        Musik        Tommy Vercetti     Vercetti_031   
Ueli Mäder             m   Soziologie            Ueli Mäder                -   
Ursula Hauser          w        Kunst                     -                -   
Ursula Rodel           w         Mode          Ursula Rodel                -   
Zora del Buono         w    Literatur   

,Name,Twitter
0,Andreas Vollenweider,0.007110
1,Yello,0.011185
2,Christoph-M. Mueller,0.008882
3,Joerg Scheller,0.983572
4,Hazel Brugger,0.731348
5,Rolf Dobelli,0.000224
6,Pedro Lenz,0.000000
7,Katja Rost,0.156400
8,Stress Officiel,0.021838
9,Riklin-Brüder,0.064496


In [ ]:
#Merge with tl again, to get the original name of the thoughtleader and not the name of the wikipedia page
final_twitter = pd.merge(df, combined, on='Name', how="left")

final_twitter.drop(columns=['Bereich', 'Wikipedia', 'Google Search Results', "Twitter Verifiziert?", "m/w"], inplace=True)
final_twitter

final_twitter.to_csv("Thoughtleader_Twitter.csv", encoding='utf-8')

In [18]:
#TODO - half of the people in our Griffin file are missing...
twitter[twitter['id'] == "PatKopViolin"]

,id,Name,followers_count,degree,betweenness,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg


In [4]:
twitter_list = []

path = r'./Time' # use your path
all_files = glob.glob(path + "/*.parquet")

for filename in all_files:
    df = pd.read_parquet(filename, engine='pyarrow')
    twitter_list.append(df)
    
#twitter = pd.read_parquet('nodes_1.parquet', engine='pyarrow')
twitter = pd.concat(twitter_list)

In [9]:
twitter

,id,time_,messages_sent,messages_received,out_degree,in_degree,degree,messages_total,contribution_index,reach_2_out,reach_2,sentiment_avg,emotionality_avg,complexity_avg,stress,happiness,betweenness
0,johannesvogel,1582934400-1583539200,1,2,3,3,6,3,-0.33,64,64,0.101458,0.214637,10.410565,0.101458,0.101458,19.666667
1,katja_rost,1582934400-1583539200,2,2,4,2,6,4,0.00,65,64,0.626417,0.166503,8.405164,0.313208,0.313208,126.000000
2,neilhimself,1582934400-1583539200,0,1,0,1,1,1,-1.00,0,3,NaN,NaN,NaN,NaN,NaN,0.000000
3,schwedenhappen_,1582934400-1583539200,0,1,0,1,1,1,-1.00,0,2,NaN,NaN,NaN,NaN,NaN,0.000000
4,uweschneidewind,1582934400-1583539200,1,2,3,3,6,3,-0.33,64,64,0.002116,0.219577,9.873338,0.002116,0.002116,29.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14821,ewaming,1566000000-1566604800,0,1,0,1,1,1,-1.00,0,2,NaN,NaN,NaN,NaN,NaN,0.000000
14822,djvde,1566000000-1566604800,0,13,0,4,4,13,-1.00,0,5,NaN,NaN,NaN,NaN,NaN,0.000000
14823,floriangathmann,1566000000-1566604800,0,7,0,2,2,7,-1.00,0,5,NaN,NaN,NaN,NaN,NaN,0.000000
14824,vforman,1566000000-1566604800,0,1,0,1,1,1,-1.00,0,2,NaN,NaN,NaN,NaN,NaN,0.000000


In [10]:
print(type(twitter["time_"]))

<class 'pandas.core.series.Series'>


In [11]:
twitter.dtypes

id                     object
time_                  object
messages_sent           int64
messages_received       int64
out_degree              int64
in_degree               int64
degree                  int64
messages_total          int64
contribution_index    float64
reach_2_out             int64
reach_2                 int64
sentiment_avg         float64
emotionality_avg      float64
complexity_avg        float64
stress                float64
happiness             float64
betweenness           float64
dtype: object

In [21]:
pd.to_datetime(1582934400 1583539200, unit='h')

SyntaxError: invalid syntax (<ipython-input-21-b074e092fb4a>, line 1)